In [3]:
import numpy as np
import pandas as pd

In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:
#1.Using numpy functions, multiply the following arrays using matrix multiplication.
 #a=np.arange(4).reshape(2,2)
# b=np.arange(6).reshape(3,2)
a=np.arange(4).reshape(2,2)
b=np.arange(6).reshape(3,2)
b_trps=np.transpose(b)
print(a)
print(b_trps)
c=a @ b_trps
print(c)

[[0 1]
 [2 3]]
[[0 2 4]
 [1 3 5]]
[[ 1  3  5]
 [ 3 13 23]]


In [21]:
OrganD_df=pd.read_excel('OrganDonor_Consolidated.xlsx')


In [26]:
OrganD_df

,PatientID,age,gender,race,causeofdeath,mechanism_of_death,brain_death,time_asystole,time_brain_death
0,OPO1_P155959,54.0,M,White,Unknown,Unknown,0,2036-01-02 18:26:00,NaT
1,OPO1_P297347,78.0,M,Hispanic,Unknown,Unknown,0,2031-12-24 14:06:00,NaT
2,OPO1_P399316,57.0,M,Hispanic,Anoxia,Unknown,0,2034-05-11 00:00:00,NaT
3,OPO1_P246407,78.0,F,Black,cerebrovascular accident,Stroke,0,2035-06-14 14:18:00,NaT
4,OPO1_P187209,70.0,F,White,Unknown,Unknown,0,2034-02-18 18:50:00,NaT
...,...,...,...,...,...,...,...,...,...
133096,OPO6_P54946,54.0,F,Black,Anoxia,Natural Causes,0,2032-09-16 02:34:00,NaT
133097,OPO6_P630991,63.0,M,Black,cerebrovascular accident,Stroke,0,2034-06-06 22:14:00,NaT
133098,OPO6_P8748,28.0,M,White,Anoxia,Drug Intoxication,0,NaT,NaT
133099,OPO6_P5461,85.0,F,White,Anoxia,Cardiovascular,0,NaT,NaT


In [22]:
DataD_df=pd.read_excel('DataDictionary.xlsx')

In [32]:
DataD_df

,Field,Description,Type,Notes
0,OPO,Anonymized OPO ID,String,e.g. OPO5
1,PatientID,Anonymized Patient ID,String,"""e.g. """"OPO5_P123"""""""
2,HospitalID,Referring hospital (anonymized ID),String,"""e.g. """"OPO5_H12"""""""
3,Age,Exact patient age,Integer,"If exact patient age was over 89, the age is l..."
4,Gender,Patient sex/gender,String,M or F
5,Race,Patient sex/gender,String,"Five categories (Black/Af-Am, White/Caucasian,..."
6,Tissue_Referral,Was patient also referred for tissue donation?,Boolean,1- referred for tissue donation\n0-not referre...
7,Eye_Referral,Was patient also referred for cornea donation?,Boolean,1- referred for eye donation\n0-not referred f...
8,causeofdeath,Cause of death,String,Categorical variable
9,mechanism_of_death,UNOS defined mechanism of death,String,Categorical variable based on UNOS definitions


In [61]:
#2.On Average, what is the time that passes between approach and procurement?
sub_OrganD_df=OrganD_df[['time_approached','time_procured']]


KeyError: "None of [Index(['time_approached', 'time_procured'], dtype='object')] are in the [columns]"

In [9]:
df_donor_details=pd.read_excel('OrganDonor_Consolidated.xlsx',sheet_name='DonorDetails')
df_donor_details

,PatientID,age,gender,race,causeofdeath,mechanism_of_death,brain_death,time_asystole,time_brain_death
0,OPO1_P155959,54.0,M,White,Unknown,Unknown,0,2036-01-02 18:26:00,NaT
1,OPO1_P297347,78.0,M,Hispanic,Unknown,Unknown,0,2031-12-24 14:06:00,NaT
2,OPO1_P399316,57.0,M,Hispanic,Anoxia,Unknown,0,2034-05-11 00:00:00,NaT
3,OPO1_P246407,78.0,F,Black,cerebrovascular accident,Stroke,0,2035-06-14 14:18:00,NaT
4,OPO1_P187209,70.0,F,White,Unknown,Unknown,0,2034-02-18 18:50:00,NaT
...,...,...,...,...,...,...,...,...,...
133096,OPO6_P54946,54.0,F,Black,Anoxia,Natural Causes,0,2032-09-16 02:34:00,NaT
133097,OPO6_P630991,63.0,M,Black,cerebrovascular accident,Stroke,0,2034-06-06 22:14:00,NaT
133098,OPO6_P8748,28.0,M,White,Anoxia,Drug Intoxication,0,NaT,NaT
133099,OPO6_P5461,85.0,F,White,Anoxia,Cardiovascular,0,NaT,NaT


In [19]:
#7.Make Patient ID as index to the table.


subdf_donor_details=df_donor_details['PatientID']
column_name=['age','gender','race','causeofdeath','mechanism_of_death','brain_death','time_asystole','time_brain_death']
subdf_donor_details
s1=pd.DataFrame(data=df_donor_details,index=subdf_donor_details,columns=column_name)
s1

,age,gender,race,causeofdeath,mechanism_of_death,brain_death,time_asystole,time_brain_death
PatientID,,,,,,,,
OPO1_P155959,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
OPO1_P297347,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
OPO1_P399316,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
OPO1_P246407,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
OPO1_P187209,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...
OPO6_P54946,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
OPO6_P630991,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
OPO6_P8748,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [51]:
#8.Create a dictionary with the Age and Cause of Death for any 5 random donors.
 #Access any one cause of death using the donors age using the get() function
dic={
    "PatientID":['OPO1_P155959','OPO1_P297347','OPO1_P399316','OPO1_P246407','OPO1_P187209'],
    "age":[54,78,57,78,70],
    "causeofdeath":['Unknown','Unknown','Anoxia','cerebrovascular accident','Unknown']
}
print(dic)


{'PatientID': ['OPO1_P155959', 'OPO1_P297347', 'OPO1_P399316', 'OPO1_P246407', 'OPO1_P187209'], 'age': [54, 78, 57, 78, 70], 'causeofdeath': ['Unknown', 'Unknown', 'Anoxia', 'cerebrovascular accident', 'Unknown']}


In [59]:
#8.Access any one cause of death using the donors age using the get() function

age=dic['age']
causeofdeath=dic['causeofdeath']
data=dict(zip(age,causeofdeath))
#data
print(data.get(78))

cerebrovascular accident
